In [62]:
import requests
# import polyline
import urllib3
import secrets
import pandas as pd

In [63]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Strava API Request

In [64]:
auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

print('Requesting Strava token... \n')
res = requests.post(auth_url, data=secrets.strava_payload, verify=False)
strava_access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + strava_access_token}

strava_requests_page_num = 1
all_activities = []

while True:
    strava_param = {'per_page' : 15, 'page' : strava_requests_page_num}
    strava_dataset = requests.get(activities_url, headers=header, params=strava_param).json()

    if len(strava_dataset) == 0:
        print('breaking out of Strava while loop because the response is zero, indicating no more activities.')
        break

    if all_activities:
        print('all activities is populated')
        all_activities.extend(strava_dataset)

    else:
        print('all activities is NOT populated')
        all_activities = strava_dataset

    strava_requests_page_num += 1

print('Total Activities: ', len(all_activities))

Requesting Strava token... 

all activities is NOT populated
all activities is populated
all activities is populated
all activities is populated
breaking out of Strava while loop because the response is zero, indicating no more activities.
Total Activities:  51


## Creating pandas dataframe for running data

In [65]:
run_data = pd.DataFrame(data=all_activities)

In [66]:
run_data.columns

Index(['resource_state', 'athlete', 'name', 'distance', 'moving_time',
       'elapsed_time', 'total_elevation_gain', 'type', 'sport_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'map', 'trainer', 'commute', 'manual', 'private',
       'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng',
       'average_speed', 'max_speed', 'has_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'workout_type', 'suffer_score',
       'average_watts', 'kilojoules', 'device_watts'],
      dtype='object')

In [67]:
run_data.head()

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,...,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,workout_type,suffer_score,average_watts,kilojoules,device_watts
0,2,"{'id': 8586088, 'resource_state': 1}","French Cattle Ranch Approach, Ten Sleep",1594.6,1483,1483,186.4,Hike,Hike,9344736492,...,70EA4DF3-F218-4DB7-B818-E242D7EC3F45-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN,NaN
1,2,"{'id': 8586088, 'resource_state': 1}","Approach to Downtown, Wild Iris",471.3,625,656,90.8,Hike,Hike,9327076219,...,7A3018DD-C2AB-447C-AF8E-92D86CE6B16E-activity.fit,False,0,0,False,NaN,NaN,NaN,NaN,NaN
2,2,"{'id': 8586088, 'resource_state': 1}",Ten Sleep Morning Run,2927.4,937,943,27.1,Run,Run,9294421659,...,D9326B4B-A20A-4FFC-963A-A75D2D17971D-activity.fit,False,0,0,False,0.0,NaN,NaN,NaN,NaN
3,2,"{'id': 8586088, 'resource_state': 1}",Ten Sleep Canyon Old Road Morning Run,3801.4,1185,1327,61.0,Run,Run,9294421645,...,9E9E8812-BB6B-4016-96DA-8E72047B2967-activity.fit,False,2,0,False,0.0,NaN,NaN,NaN,NaN
4,2,"{'id': 8586088, 'resource_state': 1}",Grays Arch loop w/ Rush,6222.5,2225,2271,173.6,Run,Run,9176575201,...,5AC4EBC8-8F82-40A1-A71E-464C999BD249-activity.fit,False,0,0,False,0.0,NaN,NaN,NaN,NaN


In [68]:
columns_to_drop = ['athlete', 
                   'sport_type', 
                   'workout_type',
                   'location_city',
                   'location_state',
                   'location_country', 
                   'kudos_count', 
                   'comment_count', 
                   'athlete_count', 
                   'photo_count', 
                   'trainer', 
                   'commute', 
                   'manual', 
                   'private',
                   'visibility', 
                   'flagged', 
                   'gear_id', 
                   'has_heartrate', 
                   'heartrate_opt_out', 
                   'display_hide_heartrate_option', 
                   'from_accepted_tag', 
                   'total_photo_count', 
                   'has_kudoed', 
                   'average_watts', 
                   'kilojoules',
                   'achievement_count',
                   'device_watts', 
                   'suffer_score']

run_data.drop(columns=columns_to_drop, inplace=True)

In [69]:
run_data.columns

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'id', 'start_date', 'start_date_local',
       'timezone', 'utc_offset', 'map', 'start_latlng', 'end_latlng',
       'average_speed', 'max_speed', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'pr_count'],
      dtype='object')

In [70]:
map_data = pd.json_normalize(run_data['map'])
map_data_info = pd.DataFrame(map_data)
map_data_info

,id,summary_polyline,resource_state
0,a9344736492,ifklG~krmSKc@?TCNOFIXIFGZ@NIPKb@MTG^EH?XEt@CHS...,2
1,a9327076219,qailGzmsmSLTADMB@DC?AIGDBDDRIAIDMAL^?DI?CBPv@I...,2
2,a9294421659,sn`lG`~~mSBBIUG_@K[A?OYE?Is@k@qB_@{@Om@Ys@MQOo...,2
3,a9294421645,_w_lGlyanSCMIu@S{@EGB[Ik@AYIIIU@MEEIc@CE?GOQGe...,2
4,a9176575201,ejweFrlr}NIQWOQAAEIAIKE?@FFHOOe@BGEKHSGQ?k@SOO...,2
5,a9159069458,{mwdFhl}}NEXUXo@?[^YPKCQt@eBvBeA|CY^Y|@GCQfAMD...,2
6,a9059541871,unwdFhj}}NACDG?GOa@M??HE@@EC?EHKHEAAFQHO?AACDW...,2
7,a9017048707,kowdFvm}}Nu@VuClDm@bBWTMv@MA[p@QhAOPD\m@r@[tAK...,2
8,a8985168804,}aeeFxop}NFv@AJJRB?C@G^c@b@@VGJDBABFS@?KSG??EK...,2
9,a8973585450,anwdFrk}}N?LQh@?CILa@FWf@IAGJkAbAEN?LCFMDCFORC...,2


In [71]:
run_data = pd.concat([run_data, map_data_info], axis=1)
run_data.drop(columns='map', inplace=True)

In [72]:
run_data.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,id,start_date,start_date_local,...,max_speed,elev_high,elev_low,upload_id,upload_id_str,external_id,pr_count,id,summary_polyline,resource_state
0,2,"French Cattle Ranch Approach, Ten Sleep",1594.6,1483,1483,186.4,Hike,9344736492,2023-06-25T18:18:46Z,2023-06-25T12:18:46Z,...,6.893,2436.5,2254.4,10023599286,10023599286,70EA4DF3-F218-4DB7-B818-E242D7EC3F45-activity.fit,0,a9344736492,ifklG~krmSKc@?TCNOFIXIFGZ@NIPKb@MTG^EH?XEt@CHS...,2
1,2,"Approach to Downtown, Wild Iris",471.3,625,656,90.8,Hike,9327076219,2023-06-22T18:07:08Z,2023-06-22T12:07:08Z,...,2.942,2294.1,2190.6,10004892705,10004892705,7A3018DD-C2AB-447C-AF8E-92D86CE6B16E-activity.fit,0,a9327076219,qailGzmsmSLTADMB@DC?AIGDBDDRIAIDMAL^?DI?CBPv@I...,2
2,2,Ten Sleep Morning Run,2927.4,937,943,27.1,Run,9294421659,2023-06-19T12:34:47Z,2023-06-19T06:34:47Z,...,4.750,1655.9,1583.3,9970213520,9970213520,D9326B4B-A20A-4FFC-963A-A75D2D17971D-activity.fit,0,a9294421659,sn`lG`~~mSBBIUG_@K[A?OYE?Is@k@qB_@{@Om@Ys@MQOo...,2
3,2,Ten Sleep Canyon Old Road Morning Run,3801.4,1185,1327,61.0,Run,9294421645,2023-06-17T12:57:06Z,2023-06-17T06:57:06Z,...,6.196,1645.2,1583.4,9970213502,9970213502,9E9E8812-BB6B-4016-96DA-8E72047B2967-activity.fit,2,a9294421645,_w_lGlyanSCMIu@S{@EGB[Ik@AYIIIU@MEEIc@CE?GOQGe...,2
4,2,Grays Arch loop w/ Rush,6222.5,2225,2271,173.6,Run,9176575201,2023-05-31T15:42:37Z,2023-05-31T11:42:37Z,...,7.315,392.0,255.0,9845666953,9845666953,5AC4EBC8-8F82-40A1-A71E-464C999BD249-activity.fit,0,a9176575201,ejweFrlr}NIQWOQAAEIAIKE?@FFHOOe@BGEKHSGQ?k@SOO...,2


## Conversions for Metrics

In [73]:
run_data['distance_miles'] = run_data['distance'] * 0.00062137119
run_data['moving_time_minutes'] = run_data['moving_time'] / 60
run_data['moving_time_hours'] = run_data['moving_time'] / 3600

In [74]:
average_distance_miles = run_data['distance_miles'].mean()
print("Average Distance (miles):", average_distance_miles)

average_time_minutes = run_data['moving_time_minutes'].mean()
print("Average Time Ran (minutes):", average_time_minutes)

max_distance_ran = run_data['distance_miles'].max()
print("Longest Run:", max_distance_ran, "miles")

max_duration_mintues = run_data['moving_time_minutes'].max()
max_duration_hours = run_data['moving_time_hours'].max()
print("Longest Duration:", max_duration_mintues,"minutes. Converted to hours:", max_duration_hours)

total_distance_miles = run_data['distance'].sum() * 0.00062137119
print("Total Distance Covered to the date (miles):", total_distance_miles)

Average Distance (miles): 3.29457705999853
Average Time Ran (minutes): 39.28071895424836
Longest Run: 9.618018238653 miles
Longest Duration: 117.56666666666666 minutes. Converted to hours: 1.9594444444444445
Total Distance Covered to the date (miles): 168.023430059925


are we only comparing kentucky running data for the pmrp/rrg area?
are we mapping the area?
can we make a dashboard that contains the local area and weather?
